# 5 Decorators and Special Methods 

### 5.1 Decorator Semantics

Decorators are callables that can modify the function or class they
decorate.  We'll focus on function decorators implemented as functions.

  Conceptually a decorator changes or adds to the behaviour of a
function either by modifying its arguments before the function is
called, changing its return value afterwards, or both.

  Here's a simple function:

In [ ]:
def add(first, second):
    """Return the sum of two arguments."""
    return first + second

In [ ]:
add(2, 3)

  Here's a simple function that creates and returns a function.

In [ ]:
def create_adder(first):
    def adder(second):
        return add(first, second)  # Notice first is used here!
    return adder

In [ ]:
add_2_to = create_adder(2)

In [ ]:
add_2_to(3)

  (How does this work?)

In [ ]:
add_2_to.__code__.co_freevars

In [ ]:
add_2_to.__closure__[0].cell_contents

  Next let's look at a function that accepts a function as an argument.  First the barebones version:

In [ ]:
def trace_function(func):
    def new_func(*args):
        return func(*args)
    return new_func

  Now with some print statements:

In [ ]:
def trace_function(func):
    """Add tracing before and after a function"""
    def new_func(*args):
        """The new function"""
        print(f'Called {func}{args!r}')
        result = func(*args)
        print(f'Returning {result!r}')
        return result
    return new_func

  This `trace_function` wraps the functionality of the function passed
to it by returning a new function that calls the original function,
but prints some trace information before and after.

In [ ]:
traced_add = trace_function(add)

In [ ]:
traced_add(2, 3)

  Instead of binding a new name to the new function returned from
`trace_function` we could instead re-bind the original name:

In [ ]:
add = trace_function(add)

In [ ]:
add(2, 3)

  Or we can use the decorator syntax to do that for us:  

In [ ]:
@trace_function
def add(first, second):
    """Return the sum of two arguments."""
    return first + second

In [ ]:
add(2, 3)

In [ ]:
add

In [ ]:
add.__qualname__

In [ ]:
add.__doc__

  Use `@wraps` to update the metadata of the returned function and make it more useful.

In [ ]:
import functools
def trace_function(func):
    """Add tracing before and after a function"""
    @functools.wraps(func)  # <-- Added
    def new_func(*args):
        """The new function"""
        print(f'Called {func}{args!r}')
        result = func(*args)
        print(f'Returning {result!r}')
        return result
    return new_func

In [ ]:
@trace_function
def add(first, second):
    """Return the sum of two arguments."""
    return first + second

In [ ]:
add

In [ ]:
add.__qualname__

In [ ]:
add.__doc__

  To write a decorator that takes parameters, we need to write a
function that is called with arguments and returns a decorator whose
behaviour depends on those arguments.

In [ ]:
def better_trace_function(uppercase=False):
    def trace_function(func):
        """Add tracing before and after a function"""
        @functools.wraps(func)
        def new_func(*args):
            """The new function"""
            print(f'Called {func}{args!r}')
            result = func(*args)
            print(f'Returning {result!r}')
            if uppercase:              # Two new
                return result.upper()  # lines
            return result
        return new_func
    return trace_function

In [ ]:
@better_trace_function(uppercase=False)
def concat(s, t):
    return s + t

In [ ]:
concat('spam', 'eggs')

In [ ]:
@better_trace_function(uppercase=True)
def concat(s, t):
    return s + t

In [ ]:
concat('spam', 'eggs')

  What will Python do with the following code?  Don't think about how
they're usually used.  Think about how they work in Python.

In [ ]:
def return_spam(func):
    print(f'Called return_spam({!func})'
    return 'spam'

In [ ]:
@return_spam
def add(first, second):
    """Return the sum of two arguments."""
    return first + second

  What object will the name `add` be bound to?  What is it's type?
Try to figure it out before executing these statements:

In [ ]:
add

In [ ]:
type(add)

In [ ]:
def add(first, second):
    """Return the sum of two arguments."""
    return first + second

In [ ]:
add

In [ ]:
add = return_spam(add)

In [ ]:
add

In [ ]:
type(add)

  What will Python do with the following code?

In [ ]:
def return_spam(klass):
    """Ignore the class argument and return 'spam'"""
    print(f'Called return_spam({!klass})'
    return 'spam'

In [ ]:
@return_spam
class WeirdClass2:
    pass

  What object will the name `WeirdClass` be bound to?  What is it's type?  Try
to figure it out before executing these statements:

In [ ]:
WeirdClass2

In [ ]:
type(WeirdClass2)


Name binding operations covered so far:

  - *name* `=` (assignment)
  - `del` *name* (unbinds the name)
  - `def` *name* function definition (including lambdas)
  - `def name(`*names*`):` (function execution)
  - *name*`.`*attribute_name* `=`, `__setattr__`, `__delattr__`
  - `global`, `nonlocal` (changes scope rules)
  - `except Exception as` *name*:
  - access to name bindings:
    - `globals()[`*name*`]
    - `locals()[`*name*`]
  - `import` *name*
  - `class` *name*`:`

The others:
  - `for` *name* `in ...:` (also list comprehensions and generator expressions)
  - `with expr as` *name*`:`
  - `setattr(an_object, `*name*`)`


### 5.2 Special Methods of Classes


  These "dunder" methods are related to an object's attribute access:
  - `__getattr__`
  - `__getattribute__`
  -  `__setattr__`
  - `__delattr__`

  We won't cover descriptors (`__get__`, `__set__`, and `__delete__`)
  or `__dir__`, and note that `__del__` is not a namespace operation.


  Let's look at a simple example of changing how a class handles attribute access.

In [ ]:
class UppercaseAttributes:
    """
    A class that returns uppercase values on uppercase attribute access.
    """
    # Called only if attribute access fails:
    def __getattr__(self, name):
        if name.isupper():
            if name.lower() in self.__dict__:
                return self.__dict__[
                    name.lower()].upper()
        raise AttributeError(
            f"'{self}' object has no attribute {name}.")

In [ ]:
ua = UppercaseAttributes()

In [ ]:
ua.__dict__

In [ ]:
ua.attriibute1 = 'value1'

In [ ]:
ua.attriibute1

In [ ]:
ua.__dict__

In [ ]:
ua.ATTRIIBUTE1

In [ ]:
ua.baz


  To add behaviour for specific attributes you can also use properties.

  Given what we've learned about decorators you may be able to infer a
  bit about the `setter` and `deleter` attributes of the object it
  returns.


In [ ]:
class PropertyEg:
    """@property example"""
    def __init__(self):
        self._x = 'Uninitialized'
    
    @property
    def x(self):
        """The 'x' property"""
        print('Called x getter()')
        return self._x
    
    @x.setter
    def x(self, value):
        print('Called x.setter()')
        self._x = value
    
    @x.deleter
    def x(self):
        print('Called x.deleter')
        self.__init__()  # Reinitialize _x

In [ ]:
p = PropertyEg()

In [ ]:
p._x

In [ ]:
p.x

In [ ]:
p.x = 'bar'

In [ ]:
p.x

In [ ]:
del p.x

In [ ]:
p.x

In [ ]:
p.x = 'bar'

  Usually you should just expose attributes and add properties later
if you need some measure of control or change of behaviour.


  Now let's look at an example of using `__getattr__`:

    PYTHON_RELEASES = [
        'Python 3.6.5 2018-03-28',
        'Python 3.5.5 2018-02-05',
        'Python 3.4.8 2018-02-05',
        'Python 3.3.6 2014-10-12',
        'Python 3.2.6 2014-10-12',
        'Python 3.1.5 2012-04-09',
        'Python 3.0.1 2009-02-13',
    ]

    release36 = PYTHON_RELEASES[0]

    release = ReleaseFields(release36)  # 3.6.5
    assert release.name == 'Python'
    assert release.version == '3.6.5'
    assert release.date == '2018-03-28'


  First, without `__getattr__`:

In [ ]:
class ReleaseFields:
    def __init__(self, data):
        self.data = data
    
    @property
    def name(self):
        return self.data[0:6]
    
    @property
    def version(self):
        return self.data[7:12]
    
    @property
    def date(self):
        return self.data[13:]

In [ ]:
release36 = 'Python 3.6.5 2018-03-28'

In [ ]:
release = ReleaseFields(release36)

In [ ]:
assert release.name == 'Python'
assert release.version == '3.6.5'
assert release.date == '2018-03-28'

  However, the following is easier to extend to many fields.

In [ ]:
class ReleaseFields:
    SLICES = {
        'name': slice(0, 6),
        'version': slice(7, 12),
        'date': slice(13, None)
        }
    
    def __init__(self, data):
        self.data = data
    
    def __getattr__(self, attribute):
        if attribute in self.SLICES:
            return self.data[self.SLICES[attribute]]
        raise AttributeError(
        raise AttributeError(
            f"'{self}' object has no attribute {name}.")

In [ ]:
release = ReleaseFields(release36)

In [ ]:
assert release.name == 'Python'
assert release.version == '3.6.5'
assert release.date == '2018-03-28'

  Let's confirm that trying to access an attribute that doesn't exist
fails, as it should.

In [ ]:
release.foo == 'exception'

### 5.3 Exercises: Special Methods of Classes

Try the following:

In [ ]:
class GetTracer:
    def __getitem__(self, key):
        print('Called __getitem__({type(key)} {key!r})')

In [ ]:
g = GetTracer()

In [ ]:
g[1]

In [ ]:
g[-1]

In [ ]:
g[0:3]

In [ ]:
g[0:10:2]

In [ ]:
g['Jan']

In [ ]:
g[g]

In [ ]:
m = list('abcdefghij')

In [ ]:
m[0]

In [ ]:
m[-1]

In [ ]:
m[::2]

In [ ]:
s = slice(3)

In [ ]:
m[s]

In [ ]:
m[slice(1, 3)]

In [ ]:
m[slice(0, 2)]

In [ ]:
m[slice(0, len(m), 2)]

In [ ]:
m[::2]

In [ ]:
m[:100]

In [ ]:
list(zip('abc', 'abcdef'))